In [24]:
import correlation_matrix as co
import numpy as np

In [25]:
mn_status = 0 # do not use status = -1 (2015 data for states that gave up pursuit)
mx_status = 2 # do not use status = 3 (acquire date results, not correlated to pursuit decision)
correl_min = 1e-6 # correlation values below this (or negative) will be reset to zero, so they have no effect

filename = 'historical_factors.csv'
#filename = 'postreactor_test.csv'
#filename = 'noreactor_test.csv'
data_file = open(filename, 'r')
full_matrix = np.loadtxt(data_file, skiprows=1,usecols=(2,3,4,5,6,7,8,9,10))
#full_matrix = np.loadtxt(data_file, skiprows=1,usecols=(2,3,4,5,6,7,8,9))
relevant_mask = ((full_matrix[:,0] >= mn_status) & (full_matrix[:,0] <= mx_status))
matrix = full_matrix[relevant_mask]

cor = co.Cor_matrix(matrix)
factor = np.array(cor[0,1:])[0]
f_list = ("Auth   ", "Mil_Iso", "Reactor", "En_Repr",  "Sci_Net", "Mil_Sp ", "Conflict","U_Res  ")
#f_list = ("Auth   ", "Mil_Iso", "En_Repr",  "Sci_Net", "Mil_Sp ", "Conflict","U_Res  ")

for i in range(len(factor)):
    print f_list[i], "\t", factor[i]


Auth    	0.373356632574
Mil_Iso 	0.232872309343
Reactor 	-0.547075093173
En_Repr 	0.316654400789
Sci_Net 	0.153276071934
Mil_Sp  	0.653810667907
Conflict 	0.0612444298104
U_Res   	-0.0443418129507


In [26]:
data_file = open(filename, 'r')
lines = data_file.readlines()
mn = 100
mx = 0
for line in lines[1:]:
    words = line.split('\t');
    name = words[0] + '_' + words[1]
    # skip countries acquisition scores (status=3) because they aren't correlated to anything relevant here
    # also skip -1's because the negative status may mess up the correlation and it's a marginally relevant dataset
    if ((float(words[2]) >= mn_status) and (float(words[2]) <= mx_status)):
        weight = words[3:]
        weight = [float(i) for i in weight]
        score = co.Compute_Score(weight,factor)
        if (score < mn):
            mn = score
        elif (score > mx):
            mx = score

In [27]:
data_file = open(filename, 'r')
lines = data_file.readlines()
print( 'Country \t Cal \t Bib \t Dif')
for line in lines[1:]:
    words = line.split('\t');
    name = words[0] + '_' + words[1]
    if ((float(words[2]) >= mn_status) and (float(words[2]) <= mx_status)):
        weight = words[3:]
        weight = [float(i) for i in weight]
        score = co.Compute_Score(weight,factor)
        cal_sc = round((score - mn)/(mx -mn)*30)
        print name, "\t", cal_sc, '\t', words[2],'\t' #, cal_sc-(float(words[2])*10)

Country 	 Cal 	 Bib 	 Dif
Belarus_2015 	13.0 	0.0 	
Germany_2015 	6.0 	0.0 	
Japan_2015 	7.0 	0.0 	
Armenia_2015 	13.0 	0.0 	
Belgium_2015 	0.0 	0.0 	
Bulgar_2015 	6.0 	0.0 	
Canada_2015 	1.0 	0.0 	
Czech-R_2015 	0.0 	0.0 	
Finland_2015 	6.0 	0.0 	
Hungary_2015 	1.0 	0.0 	
Kazak_2015 	8.0 	0.0 	
Lithua_2015 	9.0 	0.0 	
Mexico_2015 	5.0 	0.0 	
Nether_2015 	9.0 	0.0 	
Saudi-A_2015 	25.0 	0.0 	
Slovak_2015 	2.0 	0.0 	
Spain_2015 	2.0 	0.0 	
Ukraine_2015 	9.0 	0.0 	
UAE_2015 	24.0 	0.0 	
Sweden_2015 	3.0 	0.0 	
Switz_2015 	3.0 	0.0 	
Indones_2015 	7.0 	0.0 	
Algeria_2015 	16.0 	0.0 	
Romania_2015 	2.0 	0.0 	
Sweden_1946 	17.0 	1.0 	
Switz_1946 	14.0 	1.0 	
Indones_1965 	13.0 	1.0 	
Algeria_1983 	22.0 	1.0 	
Romania_1985 	13.0 	1.0 	
Austral_1961 	8.0 	2.0 	
Egypt_1965 	21.0 	2.0 	
Libya_1970 	24.0 	2.0 	
S-Korea_1970 	13.0 	2.0 	
Argent_1978 	18.0 	2.0 	
Brazil_1978 	12.0 	2.0 	
Iraq_1983 	27.0 	2.0 	
Iran_1985 	25.0 	2.0 	
Syria_2000 	21.0 	2.0 	
US_1942 	20.0 	2.0 	
USSR_1945 	30.0 	2.0 

In [28]:
print factor

weights = factor
weights[factor < correl_min] = 0
f_tot = weights.sum()
weights = weights/f_tot  # normalize weights to sum to one

for i in range(len(weights)):
    print f_list[i], "\t", weights[i]

[ 0.37  0.23 -0.55  0.32  0.15  0.65  0.06 -0.04]
Auth    	0.208437699671
Mil_Iso 	0.130008051932
Reactor 	0.0
En_Repr 	0.176781953588
Sci_Net 	0.0855710306482
Mil_Sp  	0.365009697831
Conflict 	0.0341915663299
U_Res   	0.0


In [29]:
from gen_fns import get_data
from hist_bench import calc_pursuit

#factor_weights = np.array([0.15, 0.1, 0.16, 0.09, 0.1,0.1, 0.15, 0.15])

countries, col_names, all_vals = get_data(filename, n_header=1, col_list=range(2,11))
status = all_vals[:,0]
raw_data = np.delete(all_vals, 0, 1)
all_pe_vals = calc_pursuit(raw_data, weights)

for i in range(len(countries)):
    print countries[i], "\t", all_pe_vals[i]


Belarus 	4.1291
Germany 	3.5929
Japan 	4.0445
Armenia 	4.341
Belgium 	1.4943
Bulgar 	1.5342
Canada 	1.8251
Czech-R 	1.5684
Finland 	2.7384
Hungary 	1.1692
Kazak 	3.664
Lithua 	1.5342
Mexico 	2.305
Nether 	3.6271
Saudi-A 	7.7396
Slovak 	1.5342
Spain 	1.5342
Ukraine 	4.7719
UAE 	7.3602
Sweden 	2.7384
Switz 	2.3392
Indones 	2.0825
Algeria 	5.4118
Romania 	1.3116
Sweden 	4.4951
Switz 	3.4001
Indones 	4.2539
Algeria 	6.3138
Romania 	5.1953
Austral 	2.7842
Egypt 	6.8609
Libya 	6.7925
S-Korea 	4.1219
Argent 	7.0438
Brazil 	3.9207
Iraq 	9.3023
Iran 	8.677
Syria 	7.0492
US 	5.6552
USSR 	9.299
UK 	5.0735
France 	6.643
China 	5.906
Israel 	5.9152
India 	6.3313
Pakist 	5.2652
S-Afric 	6.2382
N-Korea 	8.5222
US 	7.423
USSR 	9.299
UK 	6.8414
France 	6.9198
China 	8.7005
Israel 	7.7856
India 	6.588
Pakist 	7.8668
S-Afric 	7.025
N-Korea 	8.8265
Austral 	2.0884
Egypt 	5.34
Libya 	5.3334
S-Korea 	4.8429
Argent 	3.3225
Brazil 	4.0445
Iraq 	4.3068
Iran 	6.6953
Syria 	6.4394
